In [1]:
import os
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot as plt

# Load Dataset

In [2]:
def load_dataset(directory):
    X, y = [], []
    columns = ['Pelvis Accel Sensor X,mG', 'Pelvis Accel Sensor Y,mG',
       'Pelvis Accel Sensor Z,mG', 'Pelvis Rot X,', 'Pelvis Rot Y,',    
       'Pelvis Rot Z,']
    # Iterate through each subdirectory (falls and nonFalls)
    for subdir in os.listdir(directory):
        # Ignore any files in the directory
        if not os.path.isdir(os.path.join(directory, subdir)):
            continue
        # Iterate through each file in the subdirectory
        for file in os.listdir(os.path.join(directory, subdir)):
        
            filepath = os.path.join(directory, subdir, file)
            # Load the data from each CSV file
            data = pd.read_csv(filepath,usecols=columns)
            # Append data and labels to X and y lists
            X.append(data.values)  # Assuming all columns are features
            # Assign label based on subdirectory
            y.append(1 if subdir == 'fall' else 0)  # Assuming 'falls' is class 1, and 'nonFalls' is class 0
    return np.array(X), np.array(y)




# Training and Evaluation

In [3]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy,class_weights):
    verbose, epochs, batch_size = 1, 6, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(128, input_shape=(n_timesteps,n_features), return_sequences=True))
    model.add(LSTM(128))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','recall','precision'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose,class_weight = class_weights)
    preds = model.predict(testX, batch_size=batch_size, verbose=1)
    return np.argmax(preds,axis=1)

 

In [4]:
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight


# Load training data
trainX, trainy = load_dataset('./3S/train')

class_weights = compute_class_weight(class_weight = "balanced",
                                    classes = np.unique(trainy),
                                    y = trainy)

class_weights = dict(enumerate(class_weights))


trainy = to_categorical(trainy)


# Load testing data
testX, testy = load_dataset('./3S/test')
testy = to_categorical(testy)
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 6))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 6))


true_labels = [np.argmax(label) for label in testy]
preds = evaluate_model(trainX, trainy, testX, testy,class_weights)

print("Accuracy: ",  accuracy_score(true_labels, preds))
print("Precision: ", precision_score(true_labels, preds))
print("Recall: ", recall_score(true_labels, preds))
print("F1: ", f1_score(true_labels, preds))
print(confusion_matrix(true_labels,preds))




c:\Users\mjhil\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/6
47/47 ━━━━━━━━━━━━━━━━━━━━ 17s 274ms/step - accuracy: 0.5420 - loss: 0.6986 - precision: 0.5339 - recall: 0.5938
Epoch 2/6
47/47 ━━━━━━━━━━━━━━━━━━━━ 15s 313ms/step - accuracy: 0.6546 - loss: 0.6681 - precision: 0.6537 - recall: 0.6366
Epoch 3/6
47/47 ━━━━━━━━━━━━━━━━━━━━ 15s 319ms/step - accuracy: 0.6543 - loss: 0.6490 - precision: 0.6768 - recall: 0.5647
Epoch 4/6
47/47 ━━━━━━━━━━━━━━━━━━━━ 15s 320ms/step - accuracy: 0.7119 - loss: 0.6140 - precision: 0.7164 - recall: 0.7035
Epoch 5/6
47/47 ━━━━━━━━━━━━━━━━━━━━ 16s 343ms/step - accuracy: 0.4771 - loss: 0.7145 - precision: 0.4879 - recall: 0.5248
Epoch 6/6
47/47 ━━━━━━━━━━━━━━━━━━━━ 15s 311ms/step - accuracy: 0.7774 - loss: 0.6401 - precision: 0.7699 - recall: 0.7809
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 126ms/step
Accuracy:  0.8324022346368715
Precision:  0.525
Recall:  0.65625
F1:  0.5833333333333334
[[256  38]
 [ 22  42]]
